In [0]:
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60104384.dfs.core.windows.net",
    "pZcTKMsN83zedQA3LcYU2xsID/DzMx//bY0Is0yuMyWoXNoHgx1Rb17QRSzW9ksEoZu8KqCQPtOU+ASto/2ySQ=="
)


In [0]:
curated = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/curated_reviews/"
)

curated.printSchema()
curated.show(5, truncate=False)


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: long (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: date (nullable = true)

+--------------------------------+--------+-----------------------------------------+---------+----------------+--------------------------------+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# Cell 2: Data Cleaning and Transformation (Final Version)


from pyspark.sql.functions import col, trim, lower, split, size, when, round

df = curated

# 1) Standardize data types
df = (
    df
    .withColumn("review_id", col("review_id").cast("string"))
    .withColumn("book_id", col("book_id").cast("string"))
    .withColumn("author_id", col("author_id").cast("string"))
    .withColumn("user_id", col("user_id").cast("string"))
    .withColumn("rating", col("rating").cast("double"))
    .withColumn("n_votes", col("n_votes").cast("int"))
)

# 2) Handle missing or invalid data
df = (
    df
    .withColumn("n_votes", when(col("n_votes").isNull(), 0).otherwise(col("n_votes")))
    .withColumn("language", when(col("language").isNull(), "unknown").otherwise(col("language")))
    .withColumn("review_text", when(col("review_text").isNull(), "").otherwise(col("review_text")))
    .filter(
        col("rating").isNotNull() &
        col("review_text").isNotNull() &
        col("book_id").isNotNull() &
        col("author_id").isNotNull()
    )
)

# 3) Keep ratings between 1 and 5
df = df.filter((col("rating") >= 1.0) & (col("rating") <= 5.0))

# 4) Trim and normalize text fields
df = (
    df
    .withColumn("title", trim(col("title")))
    .withColumn("author_name", trim(col("author_name")))
    .withColumn("review_text", trim(col("review_text")))
    .withColumn("language", lower(trim(col("language"))))
)

# 5) Compute review length (in words) and remove short reviews
df = df.withColumn("review_length_words", size(split(col("review_text"), r"\s+")))
df = df.filter(col("review_length_words") >= 10)

# 6) Remove duplicates
df = df.dropDuplicates(["review_id"])
df = df.dropDuplicates(["user_id", "book_id"])

print("✅ Cleaning transformations completed successfully!")




✅ Cleaning transformations completed successfully!


In [0]:

# Cell 3: Save Cleaned Data to Gold Layer


output_path = "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/features_v1/"

df.write.format("delta").mode("overwrite").save(output_path)

print("✅ Cleaning complete! Data saved to:", output_path)
print("Total rows after cleaning:", df.count())

df.printSchema()
df.show(10, truncate=False)


✅ Cleaning complete! Data saved to: abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/features_v1/
Total rows after cleaning: 13557703
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- review_length_words: integer (nullable = false)

+--------------------------------+--------+---------------------------------------------------+---------+--------------------+--------------------------------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------